In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#starting from Kaggle sales dataset daily (https://www.kaggle.com/datasets/milanzdravkovic/pharma-sales-data)

import pandas as pd

path = '/content/drive/MyDrive/pharma_sales/salesdaily.csv'
data = pd.read_csv(path)
print(data.head())

      datum  M01AB  M01AE  N02BA  N02BE  N05B  N05C   R03  R06  Year  Month  \
0  1/2/2014    0.0   3.67    3.4  32.40   7.0   0.0   0.0  2.0  2014      1   
1  1/3/2014    8.0   4.00    4.4  50.60  16.0   0.0  20.0  4.0  2014      1   
2  1/4/2014    2.0   1.00    6.5  61.85  10.0   0.0   9.0  1.0  2014      1   
3  1/5/2014    4.0   3.00    7.0  41.10   8.0   0.0   3.0  0.0  2014      1   
4  1/6/2014    5.0   1.00    4.5  21.70  16.0   2.0   6.0  2.0  2014      1   

   Hour Weekday Name  
0   248     Thursday  
1   276       Friday  
2   276     Saturday  
3   276       Sunday  
4   276       Monday  


In [16]:
#add a city where it was sold
import random

it_cities = pd.read_csv('/content/drive/MyDrive/pharma_sales/it.csv')
print(it_cities.head())

      city      lat      lng country iso2 admin_name  capital  population  \
0     Rome  41.8933  12.4828   Italy   IT      Lazio  primary     2748109   
1    Milan  45.4669   9.1900   Italy   IT   Lombardy    admin     1354196   
2   Naples  40.8333  14.2500   Italy   IT   Campania    admin      913462   
3    Turin  45.0792   7.6761   Italy   IT   Piedmont    admin      841600   
4  Palermo  38.1111  13.3517   Italy   IT    Sicilia    admin      630167   

   population_proper  
0            2748109  
1            1354196  
2             913462  
3             841600  
4             630167  


In [14]:
#randomly sample the cities with replacement to add it to the pharmaceutical sales data
sampled_cities = it_cities.sample(n=len(data), replace=True).reset_index(drop=True)
result_df = pd.concat([data.reset_index(drop=True), sampled_cities.reset_index(drop=True)], axis=1)
print(result_df.head())

      datum  M01AB  M01AE  N02BA  N02BE  N05B  N05C   R03  R06  Year  ...  \
0  1/2/2014    0.0   3.67    3.4  32.40   7.0   0.0   0.0  2.0  2014  ...   
1  1/3/2014    8.0   4.00    4.4  50.60  16.0   0.0  20.0  4.0  2014  ...   
2  1/4/2014    2.0   1.00    6.5  61.85  10.0   0.0   9.0  1.0  2014  ...   
3  1/5/2014    4.0   3.00    7.0  41.10   8.0   0.0   3.0  0.0  2014  ...   
4  1/6/2014    5.0   1.00    4.5  21.70  16.0   2.0   6.0  2.0  2014  ...   

   Weekday Name                city      lat      lng  country  iso2  \
0      Thursday       Torre di Faro  38.2592  15.6425    Italy    IT   
1        Friday       Reggio Emilia  44.7000  10.6333    Italy    IT   
2      Saturday              Merine  40.3325  18.2261    Italy    IT   
3        Sunday    Torre Annunziata  40.7500  14.4500    Italy    IT   
4        Monday  San Mauro Torinese  45.1000   7.7667    Italy    IT   

       admin_name capital population population_proper  
0             NaN     NaN       2500           

In [30]:
#preprocessing: treating null values
null_values = result_df.isnull().sum()
print(null_values[null_values > 0])

capital    1705
dtype: int64


In [28]:
#Torre di Faro non ha assegnato nessuna regione
result_df[result_df['admin_name'].isnull()]['city'].unique()
#ma è in Sicilia quindi gliel'assegno
result_df['admin_name'] = result_df['admin_name'].fillna('Sicilia')

array(['Torre di Faro'], dtype=object)

In [37]:
# we won't use the capital
print(result_df['capital'].isnull().sum()/len(result_df))
print(result_df['capital'].unique()) #we won't really use it so we can drop it
result_df.drop(columns = ['capital'], inplace=True)

0.8095916429249762
[nan 'minor' 'admin' 'primary']


In [41]:
#I see the population and population_proper columns and they seem to have the same value
#so I check when the two differ
test_df = result_df[result_df['population']!=result_df['population_proper']]
print(len(test_df)/len(result_df))
#they are different a very marginal proportion of the dataset so I drop the population_proper column

0.0033238366571699905


In [42]:
result_df.drop(columns = ['population_proper'], inplace=True)

In [50]:
result_df.columns

Index(['datum', 'M01AB', 'M01AE', 'N02BA', 'N02BE', 'N05B', 'N05C', 'R03',
       'R06', 'Year', 'Month', 'Hour', 'Weekday Name', 'city', 'lat', 'lng',
       'country', 'iso2', 'admin_name', 'population'],
      dtype='object')

In [51]:
#unpivot the product and the unit sold (melting the dataframe)

# columns to keep as identifiers
id_vars = ["datum", "Year", "Month", "Hour", "Weekday Name", "city", "lat", "lng", "country", "iso2", "admin_name", "population"]

# columns to unpivot
value_vars = ["M01AB", "M01AE", "N02BA", "N02BE", "N05B", "N05C", "R03", "R06"]

# melt the DataFrame
melted_df = pd.melt(result_df, id_vars=id_vars, value_vars=value_vars, var_name="product", value_name="unit_sold")

# Display the resulting DataFrame
print(melted_df)


           datum  Year  Month  Hour Weekday Name                 city  \
0       1/2/2014  2014      1   248     Thursday        Torre di Faro   
1       1/3/2014  2014      1   276       Friday        Reggio Emilia   
2       1/4/2014  2014      1   276     Saturday               Merine   
3       1/5/2014  2014      1   276       Sunday     Torre Annunziata   
4       1/6/2014  2014      1   276       Monday   San Mauro Torinese   
...          ...   ...    ...   ...          ...                  ...   
16843  10/4/2019  2019     10   276       Friday  Cavenago di Brianza   
16844  10/5/2019  2019     10   276     Saturday    Mugnano di Napoli   
16845  10/6/2019  2019     10   276       Sunday                Cantù   
16846  10/7/2019  2019     10   276       Monday             Cagliari   
16847  10/8/2019  2019     10   190      Tuesday             Nerviano   

           lat      lng country iso2      admin_name  population product  \
0      38.2592  15.6425   Italy   IT         Si

In [54]:
# add hypothetycal price, cost and margin for each product

# hypothetical info with product price, cost and margins
product_info = {
    "product": ["M01AB", "M01AE", "N02BA", "N02BE", "N05B", "N05C", "R03", "R06"],
    "price": [12, 14, 10, 16, 18, 15, 20, 22],
    "cost": [9, 10, 7, 12, 14, 12, 16, 17],
    "margin": [3, 4, 3, 4, 4, 3, 4, 5]
}
product_info_df = pd.DataFrame(product_info)

#add the info to the main df
merged_df = pd.merge(melted_df, product_info_df, on="product", how = "left")
print(merged_df)

           datum  Year  Month  Hour Weekday Name                 city  \
0       1/2/2014  2014      1   248     Thursday        Torre di Faro   
1       1/3/2014  2014      1   276       Friday        Reggio Emilia   
2       1/4/2014  2014      1   276     Saturday               Merine   
3       1/5/2014  2014      1   276       Sunday     Torre Annunziata   
4       1/6/2014  2014      1   276       Monday   San Mauro Torinese   
...          ...   ...    ...   ...          ...                  ...   
16843  10/4/2019  2019     10   276       Friday  Cavenago di Brianza   
16844  10/5/2019  2019     10   276     Saturday    Mugnano di Napoli   
16845  10/6/2019  2019     10   276       Sunday                Cantù   
16846  10/7/2019  2019     10   276       Monday             Cagliari   
16847  10/8/2019  2019     10   190      Tuesday             Nerviano   

           lat      lng country iso2      admin_name  population product  \
0      38.2592  15.6425   Italy   IT         Si

In [61]:
merged_df['product'][16843].startswith('R')

True

In [62]:
#group the pharmaceutical brand name into organizational areas eventually (keep the dataset transactional)
# M01AB  M01AE = general medicine
# N02BE  N05B  N05C = neurology
# R03  R06 = rare diseases

# Define the add_division function
def add_division(product):
    if product.startswith('M'):
        return 'General Medicine'
    elif product.startswith('N'):
        return 'Neurology'
    elif product.startswith('R'):
        return 'Rare Diseases'
    else:
        return "ValueError: product not among available ones"

# Apply the add_division function to add the 'division' column
merged_df['division'] = merged_df['product'].apply(add_division)
print(merged_df)

           datum  Year  Month  Hour Weekday Name                 city  \
0       1/2/2014  2014      1   248     Thursday        Torre di Faro   
1       1/3/2014  2014      1   276       Friday        Reggio Emilia   
2       1/4/2014  2014      1   276     Saturday               Merine   
3       1/5/2014  2014      1   276       Sunday     Torre Annunziata   
4       1/6/2014  2014      1   276       Monday   San Mauro Torinese   
...          ...   ...    ...   ...          ...                  ...   
16843  10/4/2019  2019     10   276       Friday  Cavenago di Brianza   
16844  10/5/2019  2019     10   276     Saturday    Mugnano di Napoli   
16845  10/6/2019  2019     10   276       Sunday                Cantù   
16846  10/7/2019  2019     10   276       Monday             Cagliari   
16847  10/8/2019  2019     10   190      Tuesday             Nerviano   

           lat      lng country iso2      admin_name  population product  \
0      38.2592  15.6425   Italy   IT         Si

In [64]:
# Save the final DataFrame to a CSV file in Google Drive
output_path = '/content/drive/My Drive/pharma_sales/sales_augmented.csv'
merged_df.to_csv(output_path, index=False)

In [65]:
merged_df.columns

Index(['datum', 'Year', 'Month', 'Hour', 'Weekday Name', 'city', 'lat', 'lng',
       'country', 'iso2', 'admin_name', 'population', 'product', 'unit_sold',
       'price', 'cost', 'margin', 'division'],
      dtype='object')